In [86]:
import time
import re
import pandas as pd
from tqdm.notebook import tqdm

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [111]:
# Initiate a Chrome web driver
driver = webdriver.Chrome()

In [112]:
# Key page URLs
login_page_url = 'https://monash.csod.com/samldefault.aspx'
class_list_page_url = 'https://monash.csod.com/LMS/BrowseTraining/BrowseTraining.aspx'

# Login by Okta

In [113]:
driver.get(login_page_url)

# Load online class list page

In [114]:
def click_button(button):
    while True:
        try:
            button.click()
            break
        except:
            time.sleep(1)

In [115]:
def loading_class_list_page():
    print('[....] Loading the online class list page', end='\r')
    driver.get(class_list_page_url)
    while True:
        # Check if the clear type button has been rendered
        elements = driver.find_elements_by_xpath("//a[@data-bind='click: vmLoType.clearSelectedLoType']")
        if len(elements) > 0:
            print('[Done]')
            break
        time.sleep(1)

In [116]:
def get_class_list_page():
    loading_class_list_page()
    # Choose type: Online classes
    print('[....] Clear classes type', end='\r')
    clear_type_button = driver.find_element_by_xpath("//a[@data-bind='click: vmLoType.clearSelectedLoType']")
    click_button(clear_type_button)
    print('[Done]')
    print('[....] Choose online course type', end='\r')
    online_course_button = driver.find_element_by_xpath("//span[@role='presentation']/a[@data-original-title='Online course']")
    click_button(online_course_button)
    print('[Done]')
    # Choose view: List
    print('[....] Change to list view', end='\r')
    list_view_button = driver.find_element_by_xpath("//a[@title='List view']")
    click_button(list_view_button)
    print('[Done]')

In [117]:
get_class_list_page()

[Done] Loading the online class list page
[Done] Clear classes type
[Done] Choose online course type
[Done] Change to list view


# Get the total numebr of classes

In [94]:
def get_num_classes():
    num_results_span = driver.find_element_by_xpath("//h1/span[@data-bind='text: totalResultString(), visible: !isLoadingResults()']")
    num_classes = int(num_results_span.text.split()[0])
    return num_classes

In [95]:
num_classes = get_num_classes()
print('Total number of available classes:', num_classes)

Total number of available classes: 89


# Fetch classes data

In [97]:
def get_class_name_and_link(i):
    # Load the online classes list
    print('[....] Searching for the class link', end='\r')
    class_list = driver.find_elements_by_xpath("//h3/a[@class='ellipsis']")
    while i >= len(class_list):
        # Scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        class_list = driver.find_elements_by_xpath("//h3/a[@class='ellipsis']")
    print('[Done]')
    class_name = class_list[i].text
    class_link = class_list[i]
    return class_name, class_link

In [124]:
def loading_class_page(page_link):
    # Loading the class page
    print('[....] Loading the class page', end='\r')
    page_link.click()
    while True:
        # Check if the class title has been rendered
        elements = driver.find_elements_by_xpath("//h1[@data-test='LD_Title']")
        if len(elements) > 0:
            print('[Done]')
            break
        time.sleep(1)
    # Click the show more button
    print('[....] Click the show more button', end='\r')
    show_more_button = driver.find_element_by_xpath("//button[@class='sc-gtfDJT eikTzr sc-hgHYgh qtyAb']")
    click_button(show_more_button)
    print('[Done]')

In [125]:
def fetch_credit_type():
    print('[....] Fetch credit type data', end='\r')
    details_text = driver.find_element_by_xpath("//section[@data-test='Details_Section']").text
    if "Professionalism" in details_text:
        credit_type = "Professional"
    elif "Excellence in Research & Teaching" in details_text:
        credit_type = "Research"
    else:
        credit_type = "Unsure"
    print('[Done]')
    return credit_type

In [133]:
def fetch_credit_hours():
    print('[....] Fetch credit hours data', end='\r')
    credit_hours = driver.find_elements_by_xpath("//div/div/dl/div/dd[@data-test='LD_Meta_Value']/div")[-1].text
    print('[Done]')
    return credit_hours

In [127]:
def fetch_additional_hours():
    print('[....] Fetch additional hours data', end='\r')
    details_text = driver.find_element_by_xpath("//section[@data-test='Details_Section']").text
    re_result = re.findall(r'additional ?(\d\.?\d?)', details_text)
    additional_hours = re_result[0] if len(re_result) > 0 else '0'
    print('[Done]')
    return additional_hours

In [128]:
def fetch_is_linkedin():
    print('[....] Fetch facilitator data', end='\r')
    details_text = driver.find_element_by_xpath("//section[@data-test='Details_Section']").text
    if "LinkedIn Learning" in details_text:
        is_linkedin = 'Yes'
    else:
        is_linkedin = 'No'
    print('[Done]')
    return is_linkedin

In [134]:
# Initiate the dataframe
df = pd.DataFrame(columns=['Class Name', 'Type', 'Graduate Education Hour', 'Additional Hour', 'Linkedin', 'URL'])
# Fetching data
for i in tqdm(range(num_classes)):
    print('- Starting to fetch No.%d class data ...' % i)
    # Start from the list page
    if 'BrowseTraining' not in driver.current_url:
        get_class_list_page()
    # Only consider the MGR Online classes
    class_name, class_link = get_class_name_and_link(i)
    if class_name[:3] != 'MGR':
        continue
    # Load the i-th class page
    loading_class_page(class_link)
    # Fetch data
    credit_type = fetch_credit_type()
    credit_hours = fetch_credit_hours()
    additional_hours = fetch_additional_hours()
    is_linkedin = fetch_is_linkedin()
    class_page_url = driver.current_url
    # Store data into dataframe
    print('[....] Store data into dataframe', end='\r')
    df.loc[len(df)] = [class_name, credit_type, credit_hours, additional_hours, is_linkedin, class_page_url]
    print('[Done]')

- Starting to fetch No.0 class data ...
[Done] Loading the online class list page
[Done] Clear classes type
[Done] Choose online course type
[Done] Change to list view
[Done] Searching for the class link
- Starting to fetch No.1 class data ...
[Done] Searching for the class link
- Starting to fetch No.2 class data ...
[Done] Searching for the class link
- Starting to fetch No.3 class data ...
[Done] Searching for the class link
- Starting to fetch No.4 class data ...
[Done] Searching for the class link
- Starting to fetch No.5 class data ...
[Done] Searching for the class link
- Starting to fetch No.6 class data ...
[Done] Searching for the class link
- Starting to fetch No.7 class data ...
[Done] Searching for the class link
- Starting to fetch No.8 class data ...
[Done] Searching for the class link
- Starting to fetch No.9 class data ...
[Done] Searching for the class link
- Starting to fetch No.10 class data ...
[Done] Searching for the class link
- Starting to fetch No.11 class dat

[Done] Searching for the class link
[Done] Loading the class page
[Done] Click the show more button
[Done] Fetch credit type data
[Done] Fetch credit hours data
[Done] Fetch additional hours data
[Done] Fetch facilitator data
[Done] Store data into dataframe
- Starting to fetch No.32 class data ...
[Done] Loading the online class list page
[Done] Clear classes type
[Done] Choose online course type
[Done] Change to list view
[Done] Searching for the class link
[Done] Loading the class page
[Done] Click the show more button
[Done] Fetch credit type data
[Done] Fetch credit hours data
[Done] Fetch additional hours data
[Done] Fetch facilitator data
[Done] Store data into dataframe
- Starting to fetch No.33 class data ...
[Done] Loading the online class list page
[Done] Clear classes type
[Done] Choose online course type
[Done] Change to list view
[Done] Searching for the class link
[Done] Loading the class page
[Done] Click the show more button
[Done] Fetch credit type data
[Done] Fetch c

[Done] Loading the class page
[Done] Click the show more button
[Done] Fetch credit type data
[Done] Fetch credit hours data
[Done] Fetch additional hours data
[Done] Fetch facilitator data
[Done] Store data into dataframe
- Starting to fetch No.51 class data ...
[Done] Loading the online class list page
[Done] Clear classes type
[Done] Choose online course type
[Done] Change to list view
[Done] Searching for the class link
[Done] Loading the class page
[Done] Click the show more button
[Done] Fetch credit type data
[Done] Fetch credit hours data
[Done] Fetch additional hours data
[Done] Fetch facilitator data
[Done] Store data into dataframe
- Starting to fetch No.52 class data ...
[Done] Loading the online class list page
[Done] Clear classes type
[Done] Choose online course type
[Done] Change to list view
[Done] Searching for the class link
[Done] Loading the class page
[Done] Click the show more button
[Done] Fetch credit type data
[Done] Fetch credit hours data
[Done] Fetch additi

[Done] Loading the online class list page
[Done] Clear classes type
[Done] Choose online course type
[Done] Change to list view
[Done] Searching for the class link
- Starting to fetch No.71 class data ...
[Done] Searching for the class link
- Starting to fetch No.72 class data ...
[Done] Searching for the class link
- Starting to fetch No.73 class data ...
[Done] Searching for the class link
- Starting to fetch No.74 class data ...
[Done] Searching for the class link
- Starting to fetch No.75 class data ...
[Done] Searching for the class link
- Starting to fetch No.76 class data ...
[Done] Searching for the class link
- Starting to fetch No.77 class data ...
[Done] Searching for the class link
- Starting to fetch No.78 class data ...
[Done] Searching for the class link
- Starting to fetch No.79 class data ...
[Done] Searching for the class link
- Starting to fetch No.80 class data ...
[Done] Searching for the class link
- Starting to fetch No.81 class data ...
[Done] Searching for the 

In [135]:
df = df.sort_values(['Type', 'Graduate Education Hour', 'Class Name'])

In [136]:
print(len(df))
df.head()

55


,Class Name,Type,Graduate Education Hour,Additional Hour,Linkedin,URL
0,MGR Online: Body Language for Leaders,Professional,1,0,Yes,https://monash.csod.com/ui/lms-learning-detail...
5,MGR Online: Effective Listening,Professional,1,0,Yes,https://monash.csod.com/ui/lms-learning-detail...
9,MGR Online: From Idea to Business Opportunity,Professional,1,1,No,https://monash.csod.com/ui/lms-learning-detail...
10,MGR Online: Having Difficult Conversations,Professional,1,0,Yes,https://monash.csod.com/ui/lms-learning-detail...
16,MGR Online: Leading with Emotional Intelligence,Professional,1,0,Yes,https://monash.csod.com/ui/lms-learning-detail...


In [137]:
df.to_csv('MGR_classes_credits.csv', index=False)